In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import datasets,models,layers
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms

import matplotlib.pyplot as plt
import csv

import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from torch.utils.data import TensorDataset, DataLoader
import os
import shutil
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = tf.keras.datasets.cifar10

In [ ]:
(train_images, train_labels), (test_images, test_labels) = data.load_data()

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3418, 0.3126, 0.3224], std=[0.1627, 0.1632, 0.1731])
])

In [ ]:
tensor_x = torch.Tensor(train_images).permute(0,3,1,2)
tensor_y = torch.Tensor(train_labels)
tensor_x_test = torch.Tensor(test_images).permute(0,3,1,2) 
tensor_y_test = torch.Tensor(test_labels)

In [ ]:
train_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
train_loader = DataLoader(train_dataset, batch_size = 128) # create your dataloader
test_dataset = TensorDataset(tensor_x_test,tensor_y_test) # create your datset
test_loader = DataLoader(test_dataset, batch_size = 128) # create your dataloader

In [ ]:
def model_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

def set_parameter_requires_grad(model):
    for param in model.parameters():
        param.requires_grad = True

In [ ]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', pretrained=True)
device = "cuda:0"

In [ ]:
set_parameter_requires_grad(model)
model.fc = nn.Sequential(nn.Linear(512, 128), nn.ReLU(), nn.Dropout(0.4), nn.Linear(128, 10))

In [ ]:
feature_extract = False
params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized

In [ ]:
print(model_params(model))
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.1)
epochs = 10
steps = 0
running_loss = 0
train_losses = []
test_losses = []
print_every = 100

In [ ]:
for epoch in range(epochs):
    steps = 0
    for inputs, labels in train_loader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
       
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels.squeeze(1).long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if steps % print_every == 0:
            test_loss = 0
            accuracy_t = 0
            model.eval()
            with torch.no_grad():
                for inputs_val, labels_val in test_loader:
                    inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)
                    
                    logps_t = model.forward(inputs_val)
                    batch_loss = criterion(logps_t, labels_val.squeeze(1).long())
                    test_loss += batch_loss.item()
                    
                    top_p, top_class = logps_t.topk(1, dim=1)
                    equals = top_class == labels_val.view(*top_class.shape)
                    accuracy_t += torch.mean(equals.type(torch.FloatTensor)).item()
            
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(test_loader):.3f}.. "
                  f"Test accuracy: {accuracy_t/len(test_loader):.3f}")
            running_loss = 0
            model.train()
    scheduler.step()
    test_losses.append(test_loss/len(test_loader))
    train_losses.append(running_loss/len(train_loader))

In [ ]:
trial = torch.FloatTensor(1,3,256,256).to(device)

In [ ]:
path = "rn.onnx"
path_pt = "rn.pt"
torch.save(model.state_dict(), path_pt)
torch.onnx.export(model, trial, path, opset_version=11)